Assignment: Flowers Recognition <br>
Dataset Description:<br>

This dataset contains 4242 images of flowers.<br>
The data collection is based on the data flicr, google images, yandex images.<br>
You can use this datastet to recognize plants from the photo.<br>

Attribute Information:<br>
The pictures are divided into five classes: chamomile, tulip, rose, sunflower, dandelion.<br>
For each class there are about 800 photos. Photos are not high resolution, about 320x240 pixels. <br>
<b>Also explore how to resize images in tensorflow and then resize all the images to a same size. </b> <br>
This is a Multiclass Classification Problem.<br>




WORKFLOW : <br>
Load Data <br>
Split into 60 and 40 ratio.<br>
Encode labels.<br>
Create Model<br>
Compilation Step (Note : Its a Multiclass Classification problem , select loss , metrics according to it)<br>
Train the Model.<br>
If the model gets overfit tune your model by changing the units , No. of layers , epochs , add dropout layer or add Regularizer according to the need .<br>
Prediction should be > 85%<br>
Evaluation Step<br>
Prediction<br>




Data : <br>
https://drive.google.com/file/d/1-OX6wn5gA-bJpjPNfSyaYQLz-A-AB_uj/view?usp=sharing

In [35]:
import pandas as pd
import numpy as np

import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam, SGD


### IMAGE PREPROCESSING DONE AT LOCAL SYSTEM AND BROUGHT THE DATA IN SHAPE OF CSV TO COLAB - THE PREPROCESSING IS DONE USING FOLLOWING SCRIPT

#### IMAGE RESIZE AND GRESCALE SCRIPT

In [36]:
import os
# import numpy as np
from PIL import Image
from pathlib import Path

path = "images/"


# This will resise the Image to 50 x 50.
def image_resize(path,filename):
    load_img_rz = np.array(Image.open(os.path.join(path, filename)).resize((50, 50)))
    Image.fromarray(load_img_rz).save(os.path.join(path, filename))

# This will convert image into greyscale
# Warning This will replace the files
def save_as_greyscale(path,filename):
    im = np.array(Image.open(os.path.join(path, filename)).convert('L'))  # you can pass multiple arguments in single line
    gr_im = Image.fromarray(im).save(os.path.join(path,filename))



for dirpath, dirnames, filenames in os.walk(path):
    for filename in filenames:
        file_name_with_path = os.path.join(dirpath, filename)
        bn = os.path.basename(filename)
        save_as_greyscale(dirpath,filename)
        image_resize(dirpath,filename)
        print (file_name_with_path)

#### CONVERTED IMAGES INTO NUMPY ARRAY WITH LABELS FROM DIRECTORY NAME USING FOLLOIWNG SCRIPT

In [37]:
from numpy import asarray
from PIL import Image
#import pandas as pd
# import numpy as np
# import os

# Path of Images Directory
path = "images/flowers/"

# Creating a blank dataframe
df = pd.DataFrame()

# Creating a blank list
listt = []

# This function will convert image to numpy array and flatten the received array
# Function Arguments : file path and directory name as flower label
def image_to_numpy_flaten(path, fl_label):
    image = Image.open(path)
    numpydata = asarray(image)
    fdata = numpydata.flatten()
    fdata = np.append(fdata, fl_label)
    return fdata


# For loop with Os.walk
# Creating Label Name from directory name
# Calling Function image_to_numpy_flatten

for dirpath, dirnames, filenames in os.walk(path):
    for filename in filenames:
        label = dirpath.split("/")
        label = label[-1].split("\\")
        label = label[-1]
        result = image_to_numpy_flaten(os.path.join(dirpath,filename), label)
        listt.append(result)



newvar = pd.DataFrame(listt)
newvar.to_csv('flower_data.csv', mode='a', index=False, header=False)

#### LOADING CSV FILE

In [38]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [39]:
data = pd.read_csv('/content/gdrive/MyDrive/flower_data.csv', header=None, index_col=False)

In [40]:
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,2461,2462,2463,2464,2465,2466,2467,2468,2469,2470,2471,2472,2473,2474,2475,2476,2477,2478,2479,2480,2481,2482,2483,2484,2485,2486,2487,2488,2489,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499,2500
0,138,154,167,168,162,156,152,149,147,150,146,141,152,169,169,157,162,162,167,173,171,166,168,175,173,168,169,162,149,156,175,183,179,164,147,142,150,160,166,166,...,124,124,137,169,171,175,179,182,177,166,152,143,138,171,174,164,176,187,185,180,156,146,126,132,181,178,171,156,164,174,175,165,145,133,133,135,133,128,128,daisy
1,225,217,221,227,219,132,63,124,189,183,197,184,208,163,137,184,160,133,126,125,113,117,131,130,139,138,139,143,147,147,141,136,121,139,116,144,109,8,23,60,...,236,204,228,232,240,247,244,241,231,222,233,243,237,222,214,216,230,237,226,211,203,173,139,149,140,107,105,107,104,113,116,107,104,109,145,173,142,137,164,daisy
2,102,87,89,76,93,126,115,113,147,123,133,158,131,156,128,156,148,114,94,100,90,103,138,125,111,108,123,111,107,123,124,119,129,137,126,143,133,132,115,132,...,9,9,18,27,31,22,13,8,12,22,61,142,217,114,97,204,119,44,11,20,6,21,19,19,23,24,25,31,39,35,36,38,37,29,20,20,26,35,35,daisy
3,33,26,28,33,33,36,37,31,43,37,53,35,27,49,43,44,40,57,52,54,55,58,69,59,57,81,72,70,78,72,57,29,23,16,35,57,22,48,73,70,...,9,12,9,6,8,10,14,14,11,12,15,15,13,16,16,16,17,18,19,21,22,19,20,20,19,17,17,19,21,20,18,16,17,21,22,20,18,22,22,daisy
4,17,29,42,42,47,45,32,39,38,45,53,73,71,52,48,48,34,41,26,31,36,29,46,60,62,60,36,29,50,50,27,20,23,32,33,22,16,18,20,17,...,24,24,25,26,27,30,30,29,28,33,38,37,31,30,29,28,27,28,29,31,32,30,32,34,33,30,28,26,26,24,29,33,32,28,26,29,33,31,32,daisy


### Checking Null Values

In [41]:
data.isnull().sum()


0       0
1       0
2       0
3       0
4       0
       ..
2496    0
2497    0
2498    0
2499    0
2500    0
Length: 2501, dtype: int64

In [42]:
data.info

<bound method DataFrame.info of       0     1     2     3     4     5     ...  2495  2496  2497  2498  2499   2500
0      138   154   167   168   162   156  ...   133   135   133   128   128  daisy
1      225   217   221   227   219   132  ...   145   173   142   137   164  daisy
2      102    87    89    76    93   126  ...    20    20    26    35    35  daisy
3       33    26    28    33    33    36  ...    22    20    18    22    22  daisy
4       17    29    42    42    47    45  ...    26    29    33    31    32  daisy
...    ...   ...   ...   ...   ...   ...  ...   ...   ...   ...   ...   ...    ...
4318   203   217   201   197    56    96  ...    23    46   107   133   167  tulip
4319   135   132   124   123   135   147  ...    51    62    79    99   114  tulip
4320    80    79    82    89    90    86  ...    84    83    85    79    79  tulip
4321   181   201   166   153   130   183  ...    33    20    26    21    34  tulip
4322    46    93   101   123   106   140  ...    75    

In [43]:
data[0].shape # Already Flatten 

(4323,)

### Data Shuffling

In [44]:
data = data.sample(frac=1).reset_index(drop=True)

In [45]:
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,2461,2462,2463,2464,2465,2466,2467,2468,2469,2470,2471,2472,2473,2474,2475,2476,2477,2478,2479,2480,2481,2482,2483,2484,2485,2486,2487,2488,2489,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499,2500
0,172,164,157,161,167,160,133,108,131,126,138,142,113,83,79,88,105,133,116,100,107,121,87,0,9,50,101,55,71,68,87,87,72,87,45,31,42,41,55,50,...,39,37,53,74,73,78,67,73,85,84,76,62,44,39,44,55,68,79,85,84,82,78,84,80,66,45,23,29,59,43,26,40,48,49,57,56,64,55,53,rose
1,15,10,6,1,16,23,24,61,73,39,19,21,25,31,31,22,29,9,27,37,8,8,29,24,29,40,39,36,27,2,38,26,29,23,28,39,36,21,19,29,...,8,28,39,28,9,11,5,12,17,9,5,8,5,6,8,9,8,6,4,4,4,4,4,4,5,7,10,13,14,7,4,1,0,1,2,1,0,0,0,rose
2,232,231,230,229,226,223,221,220,219,218,215,214,217,220,219,216,221,218,217,222,211,203,211,206,209,195,194,199,194,196,204,206,205,206,209,209,207,205,209,214,...,133,120,161,192,109,96,140,166,169,133,128,56,64,94,78,144,122,88,115,143,168,168,139,134,132,144,129,150,172,133,119,85,134,184,155,122,91,112,142,tulip
3,94,93,92,91,90,90,90,90,89,86,82,78,77,77,78,79,80,81,82,84,85,85,86,86,89,88,87,86,85,84,83,82,81,80,78,76,75,74,74,74,...,42,39,34,27,22,20,20,19,20,21,22,23,24,23,24,25,26,27,28,27,27,28,30,32,33,33,35,38,41,42,42,43,44,46,47,48,48,51,51,daisy
4,96,96,75,37,11,8,11,9,13,8,6,11,21,27,26,23,27,34,33,21,16,19,17,10,7,11,12,10,11,14,14,11,11,10,10,12,13,13,11,8,...,74,119,162,170,180,112,66,61,69,82,103,89,67,71,88,85,78,80,70,64,80,65,63,63,62,57,51,52,58,70,69,63,53,47,46,44,40,35,36,rose


In [46]:
# Separating Input and Output Colmumns 
X = data.iloc[:, :-1].values  # Input Columns
Y = data.iloc[:, -1:].values # Output Columns 

In [47]:
X.shape

(4323, 2500)

In [48]:
Y.shape

(4323, 1)

### Splitting Data into Train, Text and Validation

In [49]:
# Total Images 4323 - Data Split

def split_train_val_test(x, y, train, val, test):
    x_rows, x_col = x.shape
    y_rows, y_col = y.shape 
    
    train_start = 0
    train_stop = int(x_rows*train)
    
    val_start = train_stop
    val_stop = int(x_rows*train)+int(x_rows*val)
    
    test_start = val_stop
    test_stop = None

    x_train = x[train_start:train_stop,] # Extracting X_Training Data 
    x_val = x[val_start:val_stop,]
    x_test = x[test_start:test_stop]
    
    y_train = y[train_start:train_stop,] # Extracting y_Training Data 
    y_val = y[val_start:val_stop,]
    y_test = y[test_start:test_stop]
    
    #Splitting Desicription 
    desc = f'''Splitting Discription:
    Training Data {int(train*100)}%
    Validation Data {int(val*100)}%
    Testing Data {int(val*100)}%
    
    Shape of Training Data {x.shape}
    Shape of Target Data {y.shape}
    
    Shape of x_train {x_train.shape}
    Shape of x_val {x_val.shape}
    Shape of x_test {x_test.shape}
     
    Shape of y_train {y_train.shape}
    Shape of y_val {y_val.shape}
    Shape of y_test {y_test.shape}
    
    '''
    
    return x_train, x_val, x_test, y_train, y_val, y_test, desc



x_train, x_val, x_test, y_train, y_val, y_test, desc = split_train_val_test(X,Y,.6,.2,.2)

print (desc)


Splitting Discription:
    Training Data 60%
    Validation Data 20%
    Testing Data 20%
    
    Shape of Training Data (4323, 2500)
    Shape of Target Data (4323, 1)
    
    Shape of x_train (2593, 2500)
    Shape of x_val (864, 2500)
    Shape of x_test (866, 2500)
     
    Shape of y_train (2593, 1)
    Shape of y_val (864, 1)
    Shape of y_test (866, 1)
    
    


In [50]:
print (x_train.shape, x_test.shape, x_val.shape)
print (y_train.shape, y_test.shape, y_val.shape)

(2593, 2500) (866, 2500) (864, 2500)
(2593, 1) (866, 1) (864, 1)


# Normalization 

In [51]:
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255
x_val = x_val.astype('float32') / 255


In [52]:
x_train # After Normalization 

array([[0.6745098 , 0.6431373 , 0.6156863 , ..., 0.2509804 , 0.21568628,
        0.20784314],
       [0.05882353, 0.03921569, 0.02352941, ..., 0.        , 0.        ,
        0.        ],
       [0.9098039 , 0.90588236, 0.9019608 , ..., 0.35686275, 0.4392157 ,
        0.5568628 ],
       ...,
       [0.14901961, 0.15686275, 0.16078432, ..., 0.11764706, 0.12941177,
        0.12156863],
       [0.18039216, 0.19607843, 0.21960784, ..., 0.5176471 , 0.5019608 ,
        0.5019608 ],
       [0.4       , 0.4       , 0.28235295, ..., 0.20392157, 0.18039216,
        0.1882353 ]], dtype=float32)

#### Label Encoding

In [53]:
from sklearn.preprocessing import LabelEncoder

y_train = np.array(y_train)
y_test = np.array(y_test)
y_val = np.array(y_val)

label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.fit_transform(y_test)
y_val = label_encoder.fit_transform(y_val)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [54]:
from keras.utils import to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
y_val = to_categorical(y_val)

#### CREATING ARCHITECTURE

In [55]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import regularizers
model = models.Sequential()
model.add(layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.001), input_shape=(50 * 50,)))
model.add(layers.Dense(5, activation='softmax'))

In [56]:
model.compile(optimizer='rmsprop', 
                loss='categorical_crossentropy',
                metrics=['accuracy'])

### Training

In [57]:
history = model.fit(x_train, y_train,epochs=75, validation_data =(x_val, y_val))



Epoch 1/75
82/82 [==============================] - 2s 21ms/step - loss: 5.3674 - accuracy: 0.2264 - val_loss: 9.1248 - val_accuracy: 0.1516
Epoch 2/75
82/82 [==============================] - 2s 20ms/step - loss: 3.0430 - accuracy: 0.2444 - val_loss: 4.9653 - val_accuracy: 0.1690
Epoch 3/75
82/82 [==============================] - 2s 19ms/step - loss: 2.6966 - accuracy: 0.2504 - val_loss: 4.6432 - val_accuracy: 0.2431
Epoch 4/75
82/82 [==============================] - 2s 19ms/step - loss: 2.7334 - accuracy: 0.2366 - val_loss: 2.6426 - val_accuracy: 0.2106
Epoch 5/75
82/82 [==============================] - 2s 19ms/step - loss: 2.4611 - accuracy: 0.2430 - val_loss: 3.9748 - val_accuracy: 0.2431
Epoch 6/75
82/82 [==============================] - 2s 19ms/step - loss: 2.4896 - accuracy: 0.2418 - val_loss: 4.9807 - val_accuracy: 0.1667
Epoch 7/75
82/82 [==============================] - 2s 19ms/step - loss: 2.5341 - accuracy: 0.2408 - val_loss: 6.2823 - val_accuracy: 0.1667
Epoch 8/75
82

### Evaluation on Test Data

In [58]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print('test_acc:', test_acc)

28/28 [==============================] - 0s 6ms/step - loss: 2.2991 - accuracy: 0.2679
test_acc: 0.2678983807563782


### Prdiction

In [59]:
pred = model.predict(x_test[0:1])
pred

array([[0.08465333, 0.78229356, 0.09792668, 0.0034993 , 0.03162714]],
      dtype=float32)

###Note: 
##### Applied Drop Out and Regularization one-by-one, but validation accuracy did not improve.